# Skydipper acceptance test

In [2]:
import json
import requests
from pprint import pprint
import random
import getpass
import LMIPy

## Create a new user in Skydipper

https://api.skydipper.com/auth/login

## Get the token

In [3]:
email = 'iker.sanchez@vizzuality.com'

In [4]:
password = getpass.getpass('Skydipper login password:')

Skydipper login password: ·········


In [5]:
payload = {
    "email": f"{email}",
    "password": f"{password}"
}

In [6]:
url = f'https://api.skydipper.com/auth/login'

headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)

OAUTH = r.json().get('data').get('token')

## Carto layer
### Register Dataset
**FAO hydrobasins dataset**

In [ ]:
payload = {"dataset": {
            "name": "FAO hydrobasins",
            "application": ["skydipper"],
            "connectorType": "rest",
            "provider": "cartodb",
            "connectorUrl": "https://wri-rw.carto.com/tables/hydrobasins_fao_fiona_merged_v01/public",
            "tableName": "hydrobasins_fao_fiona_merged_v01",
            "status": "saved",
            "published": True,
            "overwrite": False,
            "verified": False,
            "env": "production",
        }
}

In [ ]:
#Post new dataset
url = f'https://api.skydipper.com/v1/dataset'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

### Register layer 

In [ ]:
payload = {
    
            "name": "FAO hydrobasins",
            "application": [
                "skydipper"
            ],
            "iso": [],
            "provider": "cartodb",
            "default": True,
            "protected": False,
            "published": True,
            "env": "production",
            "layerConfig": {
                "account": "wri-rw",
                "body": {
                    "layers": [
                        {
                            "type": "cartodb",
                            "options": {
                                "sql": "SELECT * FROM hydrobasins_fao_fiona_merged_v01",
                                "cartocss": "#hydrobasins_fao_fiona_merged_v01 { polygon-fill: transparent; polygon-opacity: 0; polygon-comp-op: overlay; } #hydrobasins_fao_fiona_merged_v01::outline { line-width: 0.75; line-color: #B0C4DE; line-opacity: 0; [zoom<6]{line-opacity: 0;}}",
                                "cartocss_version": "2.3.0"
                            }
                        }
                    ]
                }
            },
            "legendConfig": {},
            "interactionConfig": {
                "output": [
                    {
                        "type": "string",
                        "suffix": "",
                        "property": "FAO major basin",
                        "prefix": "",
                        "format": None,
                        "column": "maj_name"
                    },
                    {
                        "type": "string",
                        "suffix": "",
                        "property": "FAO minor basin",
                        "prefix": "",
                        "format": None,
                        "column": "sub_name"
                    }
                ]
            }
    
}

In [ ]:
#Register layers
dataset_id = '640adc03-5fc4-46ff-bfcf-e622d430fed4'

url = f'https://api.skydipper.com/v1/dataset/{dataset_id}/layer/'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.url)
pprint(r.json())
layer = r.json().get('data',None)

### Query a table

In [8]:
slug = 'FAO-hydrobasins'

url = f'https://api.skydipper.com/v1/query?sql=SELECT * from {slug} limit 10'

r = requests.get(url)

pprint(r.json())

{'data': [{'cartodb_id': 1,
           'legend': 1,
           'maj_area': 2086009,
           'maj_bas': 5001,
           'maj_name': 'Amur',
           'sub_area': 104012,
           'sub_bas': 1001,
           'sub_name': 'Herlen Gol /  Hulun Nur',
           'the_geom': '...',
           'the_geom_webmercator': '...',
           'to_bas': 1019},
          {'cartodb_id': 2,
           'legend': 1,
           'maj_area': 2086009,
           'maj_bas': 5001,
           'maj_name': 'Amur',
           'sub_area': 59873,
           'sub_bas': 1002,
           'sub_name': 'Onon',
           'the_geom': '...',
           'the_geom_webmercator': '...',
           'to_bas': 1006},
          {'cartodb_id': 3,
           'legend': 1,
           'maj_area': 2086009,
           'maj_bas': 5001,
           'maj_name': 'Amur',
           'sub_area': 50635,
           'sub_bas': 1003,
           'sub_name': 'Solonchak Zun Torey / Solonchak',
           'the_geom': '...',
           'the_geom_webmer

### Check field 

In [9]:
dataset_id = '640adc03-5fc4-46ff-bfcf-e622d430fed4'

url = f'https://api.skydipper.com/v1/fields/{dataset_id}'

r = requests.get(url)

pprint(r.json())

{'fields': {'cartodb_id': {'pgtype': 'int4', 'type': 'number'},
            'legend': {'pgtype': 'int4', 'type': 'number'},
            'maj_area': {'pgtype': 'int4', 'type': 'number'},
            'maj_bas': {'pgtype': 'int4', 'type': 'number'},
            'maj_name': {'pgtype': 'text', 'type': 'string'},
            'sub_area': {'pgtype': 'int4', 'type': 'number'},
            'sub_bas': {'pgtype': 'int4', 'type': 'number'},
            'sub_name': {'pgtype': 'text', 'type': 'string'},
            'the_geom': {'dims': 2,
                         'srid': 4326,
                         'type': 'geometry',
                         'wkbtype': 'Unknown'},
            'the_geom_webmercator': {'dims': 2,
                                     'srid': 3857,
                                     'type': 'geometry',
                                     'wkbtype': 'Unknown'},
            'to_bas': {'pgtype': 'int4', 'type': 'number'}},
 'tableName': 'hydrobasins_fao_fiona_merged_v01'}


## Earth Engine layer
### Register Dataset
**[Biodiversity Intactness dataset](https://resourcewatch.org/data/explore/bio_014-Biodiversity-Intactness)**

In [23]:
payload = {"dataset": {
            "name": "bio.014 Biodiversity Intactness",
            "application": ["skydipper"],
            "type": "raster",
            "connectorType": "rest",
            "provider": "gee",
            "tableName": "users/resourcewatch/bio_014_bio_intactness",
            "status": "saved",
            "published": True,
            "overwrite": False,
            "verified": False,
            "env": "production",
        }
}

In [24]:
#Post new dataset
url = f'https://api.skydipper.com/v1/dataset'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

{'data': {'id': '94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7', 'type': 'dataset', 'attributes': {'name': 'bio.014 Biodiversity Intactness', 'slug': 'bio014-Biodiversity-Intactness', 'type': 'raster', 'subtitle': None, 'application': ['skydipper'], 'dataPath': None, 'attributesPath': None, 'connectorType': 'rest', 'provider': 'gee', 'userId': '5dc9210cbe566d0010df6752', 'connectorUrl': None, 'tableName': 'users/resourcewatch/bio_014_bio_intactness', 'status': 'pending', 'published': False, 'overwrite': False, 'verified': False, 'blockchain': {}, 'mainDateField': None, 'env': 'production', 'geoInfo': False, 'protected': False, 'legend': {'keyword': [], 'text': [], 'binary': [], 'boolean': [], 'scaled_float': [], 'half_float': [], 'float': [], 'double': [], 'byte': [], 'short': [], 'integer': [], 'nested': [], 'country': [], 'region': [], 'date': []}, 'clonedHost': {}, 'errorMessage': None, 'taskId': None, 'createdAt': '2019-11-15T10:25:30.592Z', 'updatedAt': '2019-11-15T10:25:30.650Z', 'dataLas

### Deleting Datasets

In [7]:
#Delet dataset
dataset_id = '94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7'

url = f'https://api.skydipper.com/v1/dataset/{dataset_id}'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}

r = requests.delete(url, headers=headers)
print(r.json())

{'errors': [{'status': 403, 'detail': 'Forbidden'}]}


### Register layer 

In [10]:
payload = {
    
                        "name": "2005 Biodiversity Intactness (%)",
                        "description": "Modeled average abundance of originally present species, relative to their abundance in an intact ecosystem, modeled for the year 2005. Green areas are those within safe limits for biodiversity, and red areas are those beyond proposed safe limits.",
                        "application": [
                            "skydipper"
                        ],
                        "iso": [],
                        "provider": "gee",
                        "default": True,
                        "protected": False,
                        "published": True,
                        "env": "production",
                        "layerConfig": {
                            "type": "gee",
                            "assetId": "users/resourcewatch/bio_014_bio_intactness",
                            "body": {
                                "styleType": "sld",
                                "sldValue": "<RasterSymbolizer><ColorMap type=\"ramp\" extended=\"false\" ><ColorMapEntry color=\"#a50026\" quantity=\"0.6\"  opacity=\"1\" /><ColorMapEntry color=\"#d73027\" quantity=\"0.7\" /><ColorMapEntry color=\"#f46d43\" quantity=\"0.75\" /><ColorMapEntry color=\"#fdae61\" quantity=\"0.8\"  /><ColorMapEntry color=\"#fee08b\" quantity=\"0.85\"  /><ColorMapEntry color=\"#d9ef8b\" quantity=\"0.9\" /><ColorMapEntry color=\"#a6d96a\" quantity=\"0.95\"  /><ColorMapEntry color=\"#66bd63\" quantity=\"0.975\" /><ColorMapEntry color=\"#1a9850\" quantity=\"1\" /><ColorMapEntry color=\"#006837\" quantity=\"1.5\"     /></ColorMap></RasterSymbolizer>"
                            }
                        },
                        "legendConfig": {
                            "type": "choropleth",
                            "items": [
                                {
                                    "name": "≤60",
                                    "color": "#a50026",
                                    "id": 0
                                },
                                {
                                    "name": "≤70",
                                    "color": "#d73027",
                                    "id": 1
                                },
                                {
                                    "name": "≤75",
                                    "color": "#f46d43",
                                    "id": 2
                                },
                                {
                                    "name": "≤80",
                                    "color": "#fdae61",
                                    "id": 3
                                },
                                {
                                    "name": "≤85",
                                    "color": "#fee08b",
                                    "id": 4
                                },
                                {
                                    "name": "≤90",
                                    "color": "#d9ef8b",
                                    "id": 5
                                },
                                {
                                    "name": "≤95",
                                    "color": "#a6d96a",
                                    "id": 6
                                },
                                {
                                    "name": "≤97.5",
                                    "color": "#66bd63",
                                    "id": 7
                                },
                                {
                                    "name": "≤100",
                                    "color": "#1a9850",
                                    "id": 8
                                },
                                {
                                    "name": ">100",
                                    "color": "#006837",
                                    "id": 9
                                }
                            ]
                        }
    
}

In [11]:
#Register layers
dataset_id = '94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7'

url = f'https://api.skydipper.com/v1/dataset/{dataset_id}/layer/'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.url)
pprint(r.json())
layer = r.json().get('data',None)

https://api.skydipper.com/v1/dataset/94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7/layer/
{'data': {'attributes': {'application': ['skydipper'],
                         'applicationConfig': {},
                         'dataset': '94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7',
                         'default': True,
                         'description': 'Modeled average abundance of '
                                        'originally present species, relative '
                                        'to their abundance in an intact '
                                        'ecosystem, modeled for the year 2005. '
                                        'Green areas are those within safe '
                                        'limits for biodiversity, and red '
                                        'areas are those beyond proposed safe '
                                        'limits.',
                         'env': 'production',
                         'interactionConfig': {},
        

### Query raster

In [12]:
dataset_id = '94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7'
tableName = 'users/resourcewatch/bio_014_bio_intactness'

url = f'https://api.skydipper.com/v1/query/{dataset_id}?sql=SELECT st_summarystats(rast, \'b1\', false) as x from \'{tableName}\' where ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON(\'{\"type":\"Point\",\"coordinates\":[-1.5785980224609375,41.40668586105652]}\'),4326),the_geom)'

r = requests.get(url)

pprint(r.json())

{'data': [{'cartodb_id': 1,
           'legend': 1,
           'maj_area': 2086009,
           'maj_bas': 5001,
           'maj_name': 'Amur',
           'sub_area': 104012,
           'sub_bas': 1001,
           'sub_name': 'Herlen Gol /  Hulun Nur',
           'the_geom': '...',
           'the_geom_webmercator': '...',
           'to_bas': 1019},
          {'cartodb_id': 2,
           'legend': 1,
           'maj_area': 2086009,
           'maj_bas': 5001,
           'maj_name': 'Amur',
           'sub_area': 59873,
           'sub_bas': 1002,
           'sub_name': 'Onon',
           'the_geom': '...',
           'the_geom_webmercator': '...',
           'to_bas': 1006},
          {'cartodb_id': 3,
           'legend': 1,
           'maj_area': 2086009,
           'maj_bas': 5001,
           'maj_name': 'Amur',
           'sub_area': 50635,
           'sub_bas': 1003,
           'sub_name': 'Solonchak Zun Torey / Solonchak',
           'the_geom': '...',
           'the_geom_webmer

In [17]:
dataset_id = '94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7'
tableName = 'users/resourcewatch/bio_014_bio_intactness'

url = f"https://api.skydipper.com/v1/query/{dataset_id}?sql=SELECT st_summarystats(rast, \'b1\', false) as x from \'{tableName}\' where ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON(\'{"type":"Point","coordinates":[-1.5785980224609375,41.40668586105652]}\'),4326),the_geom)"


```
https://api.skydipper.com/v1/query/94241cb4-9e91-4a1a-9fcc-993b8ac9c2b7?sql=select st_summarystats(rast, 'b1', false) as x from 'users/resourcewatch/bio_014_bio_intactness' where ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON('{"type":"Point","coordinates":[-1.5785980224609375,41.40668586105652]}'),4326),the_geom)
```

## Create a widget config and attach it to a layer/Dataset

## Create a geostore object and return it 

## Upload a CSV

In [ ]:
url = f'https://api.skydipper.com/v1/dataset/{dataset_id}/layer/'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.url)
pprint(r.json())
layer = r.json().get('data',None)

## Add metadata and vocabulary 